### Zastosowanie algorytmu k-średnich w segmentacji obrazu. 

Jednym z popularnych zastosowań algorytmu k-means jest segmentacja obrazu.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
img = cv2.imread('data/img1.jpg')  # wczytujemy obraz
img.shape  # zdjęcie kolorowe

In [ ]:
img

In [ ]:
# wyświetlamy
plt.imshow(img[:,:,::-1])  # numpy i matplotlib mają odwrotną kolejność kanałów

Zróbmy z tego ndarray trzykolumnowy dataframe, po jednej kolumnie na każdy kanał.

In [ ]:
# przygotowanie obrazu do modelu
img_data = img.reshape((-1, 3))  # zmieniamy kształt
img_data = np.float32(img_data)  # konwertujemy do float32
img_data.shape

In [ ]:
df = pd.DataFrame(data=img_data, columns=['dim1', 'dim2', 'dim3'])
df

Tym razem nie użyjemy klasy KMeans biblioteki scikit-learn, tylko funkcji kmeans biblioteki openCV

In [ ]:
# popatrzmy na nią
cv2.kmeans?

# parametry na wejściu
# data 
# K - liczba klastrów
# bestLabels - etykiety
# criteria - warunek stopu
# attempts - liczba uruchomień algorytmu

# dostajemy trzy wartości
# compactness = wss
# labels = etykiety: '1', '2', ...
# centers = centroidy

In [ ]:
# uruchamiamy
_, label, center = cv2.kmeans(
    data=img_data,  # float32 data type
    K=2,            # liczba klastrów
    bestLabels=None,
    criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0),  # kryterium zatrzymania (typ, max_iter, eps)
    attempts=10,    # liczba uruchomień algorytmu 
    flags=cv2.KMEANS_RANDOM_CENTERS
)    # określenie inicjalizacji centroidów (losowe)

center = np.uint8(center)
res = center[label.flatten()]
res = res.reshape((img.shape))
plt.imshow(res[:,:,::-1])

In [ ]:
# Napiszmy funkcję, która pozwoli nam cały ten proces zautomatyzować

def make_kmeans(img_name: str, cluster_nr: int = 2) -> None:

    # wczytanie zdjęcia
    img = cv2.imread(img_name)
    plt.imshow(img[:,:,::-1])
    plt.show()
    
    # przygotowanie zdjęcia
    img_data = img.reshape((-1, 3))
    img_data = np.float32(img_data)

    # kmeans
    _, label, center = cv2.kmeans(
        data=img_data, 
        K=cluster_nr, 
        bestLabels=None,
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0), 
        attempts=10, 
        flags=cv2.KMEANS_RANDOM_CENTERS)

    # przygotowanie do wyświetlenia
    center = np.uint8(center)
    res = center[label.flatten()]
    res = res.reshape((img.shape))
    plt.imshow(res[:,:,::-1])
    plt.show()

In [ ]:
make_kmeans("data/img1.jpg")

Do czego możemy używać ?

1. detekcja obiektów

In [ ]:
make_kmeans("data/img2.jpg")

In [ ]:
make_kmeans("data/img2.jpg", 6)

2. Kompresja obrazu

In [ ]:
make_kmeans("data/img3.jpg")

In [ ]:
# Popatrzmy od 2 do 20

# wczytanie zdjęcia
img = cv2.imread('data/img3.jpg')
plt.imshow(img[:,:,::-1])
plt.show()

# przygotowanie zdjęcia
img_data = img.reshape((-1, 3))
img_data = np.float32(img_data)

for cluster_nr in range(2, 20, 4):
    print(f"Liczba klastrów: {cluster_nr}")
    _, label, center = cv2.kmeans(
        data=img_data,  # float32 data type
        K=cluster_nr,            # liczba klastrów
        bestLabels=None,
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0),  # kryterium zatrzymania (typ, max_iter, eps)
        attempts=10,    # liczba uruchomień algorytmu 
        flags=cv2.KMEANS_RANDOM_CENTERS
    )    # określenie inicjalizacji centroidów (losowe)
    
    center = np.uint8(center)
    res = center[label.flatten()]
    res = res.reshape((img.shape))
    plt.imshow(res[:,:,::-1])
    plt.show()

3. Konwersja obrazu (wstępne przetwarzanie)

In [ ]:
make_kmeans("data/img4.png", 2)